In [1]:
import covid_19_shared as COVID_19

import pandas as pd

print("Getting population...")
pop = COVID_19.USA_population()

print("Getting list of states...")
states = COVID_19.list_of_JHUD_US_states()

print("Getting USA data...")
us_data = COVID_19.USA_Data()

print("%-30s%-10s%-20s%-10s%-16s" 
      % ("State", "New Conf.", "Conf./Popul.(%)"
         , "New Death", "Death/Popul.(%)")
     )
df = pd.DataFrame()
for state in states:
    if state in pop:
        
        stats = COVID_19.COVID_19_Stats(state, COVID_19.usa_state_data(state, us_data))
        state_pop = int(pop[state])
        stats.pops_ratio_conf_ = stats.cv_data_.Confirmed * 100 / state_pop
        stats.pops_ratio_deaths_ = stats.cv_data_.Dead * 100 / state_pop
        print("%-30s%-10d%-20f%-10d%-16f" 
              % (state
                    , int(stats.confirmed_new_[-1])
                    , stats.pops_ratio_conf_[-1]
                    , int(stats.deaths_new_[-1])
                    , stats.pops_ratio_deaths_[-1]
                   )
             )
        df = df.append(pd.DataFrame({"State" : [state], "Statistics": [stats], "Population" : [state_pop]}))
#         stats.dynamics()

df = df.set_index("State")

Getting population...
Getting list of states...
Getting USA data...
State                         New Conf. Conf./Popul.(%)     New Death Death/Popul.(%) 
Alabama                       2980      4.469222            18        0.067598        
Alaska                        593       3.154607            1         0.013658        
Arizona                       3015      4.217401            17        0.097888        
Arkansas                      2312      4.469193            4         0.073665        
California                    9450      2.720694            70        0.048867        
Colorado                      6439      3.062875            36        0.049789        
Connecticut                   2746      2.480207            11        0.132537        
Delaware                      470       3.120051            2         0.081743        
District of Columbia          159       3.102092            0         0.109186        
Florida                       6933      4.630273            73

In [5]:
df["PopulShare"] =df.Population * 100 / df.Population.sum()

df["Confirmed"] = pd.Series(index=df.index, data=[(v.cv_data_.Confirmed[-1]) for v in df["Statistics"]])
df["OneDayConf"] = pd.Series(index=df.index, data=[int(v.confirmed_new_[-1]) for v in df["Statistics"]])
df["OneWeekConfirmed"] = pd.Series(index=df.index, data=[(v.cv_data_.Confirmed[-1] - v.cv_data_.Confirmed[-8]) for v in df["Statistics"]])
df["ConfShare"] = df.Confirmed * 100 / df.Confirmed.sum()
df["NewConfShare"] = df.OneWeekConfirmed * 100 / df.OneWeekConfirmed.sum()  # using one week stats

df["Deaths"] = pd.Series(index=df.index, data=[(v.cv_data_.Dead[-1]) for v in df["Statistics"]])
df["OneDayDeaths"] = pd.Series(index=df.index, data=[int(v.deaths_new_[-1]) for v in df["Statistics"]])
df["OneWeekDeaths"] = pd.Series(index=df.index, data=[(v.cv_data_.Dead[-1] - v.cv_data_.Dead[-8]) for v in df["Statistics"]])
df["DeathsShare"] = df.Deaths * 100 / df.Deaths.sum()
df["NewDeathsShare"] = df.OneWeekDeaths * 100 / df.OneWeekDeaths.sum()

# Indicator of intensity
# "Intensity" as ratio of value share to population share
df["CasesIntencity"] = df.ConfShare / df.PopulShare
df["NewCasesIntencity"] = df.NewConfShare / df.PopulShare

df["DeathsIntencity"] = df.DeathsShare / df.PopulShare
df["NewDeathsIntencity"] = df.NewDeathsShare / df.PopulShare

# df3 = df.copy()
# del df3["Statistics"]
df3 = df[["OneWeekConfirmed"
          , "OneWeekDeaths"
          , "CasesIntencity"
          , "NewCasesIntencity"
          , "DeathsIntencity"
          , "NewDeathsIntencity"
         ]]
# dropping statistics column 
df3 = df3.sort_values('OneWeekDeaths', ascending=False)
df3

,OneWeekConfirmed,OneWeekDeaths,CasesIntencity,NewCasesIntencity,DeathsIntencity,NewDeathsIntencity
State,,,,,,
Texas,65894,739,1.208071,0.836535,1.006741,1.254019
Illinois,86417,494,1.252791,2.150060,1.086083,1.642858
Florida,37927,431,1.348428,0.643961,1.187206,0.978161
Michigan,45853,428,0.790725,1.480983,1.075531,1.847770
Indiana,35742,316,1.062532,1.759740,0.959661,2.079594
Tennessee,27314,311,1.378789,1.373970,0.776644,2.091097
Georgia,17443,297,1.262169,0.574780,1.176139,1.308154
California,50355,274,0.792321,0.431489,0.625261,0.313834
Minnesota,37032,250,1.138426,2.228841,0.698384,2.011239
